# Database Check
Validate raw + derived tables and provenance for the FINRA + Polygon pipeline.


In [2]:
from darkpool_analysis.config import load_config
from darkpool_analysis.db import get_connection
import pandas as pd

config = load_config()
conn = get_connection(config.db_path)


In [3]:
tables = conn.execute("SHOW TABLES").df()["name"].tolist()
tables


['composite_signal',
 'daily_metrics',
 'finra_otc_weekly_raw',
 'finra_short_daily_all_raw',
 'finra_short_daily_raw',
 'index_constituent_short_agg_daily',
 'lit_direction_daily',
 'options_premium_daily',
 'options_premium_summary',
 'polygon_daily_agg_raw',
 'polygon_equity_trades_raw',
 'polygon_ingestion_state',
 'scanner_daily_metrics']

In [4]:
from IPython.display import display

for table in tables:
    print(table)
    display(conn.execute(f"DESCRIBE {table}").df())
    display(conn.execute(f"SELECT COUNT(*) AS n FROM {table}").df())


composite_signal


,column_name,column_type,null,key,default,extra
0,symbol,VARCHAR,YES,None,None,None
1,date,DATE,YES,None,None,None
2,directional_score,DOUBLE,YES,None,None,None
3,pressure_score,DOUBLE,YES,None,None,None
4,participation_score,DOUBLE,YES,None,None,None
5,composite_score,DOUBLE,YES,None,None,None
6,final_label,VARCHAR,YES,None,None,None
7,data_quality,VARCHAR,YES,None,None,None
8,inference_version,VARCHAR,YES,None,None,None


,n
0,0


daily_metrics


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,symbol,VARCHAR,YES,None,None,None
2,log_buy_sell,DOUBLE,YES,None,None,None
3,short_volume,DOUBLE,YES,None,None,None
4,short_exempt_volume,DOUBLE,YES,None,None,None
5,short_total_volume,DOUBLE,YES,None,None,None
6,short_buy_volume,DOUBLE,YES,None,None,None
7,short_sell_volume,DOUBLE,YES,None,None,None
8,short_ratio,DOUBLE,YES,None,None,None
9,short_ratio_z,DOUBLE,YES,None,None,None


...

,column_name,column_type,null,key,default,extra
0,symbol,VARCHAR,NO,PRI,None,None
1,trade_date,DATE,NO,PRI,None,None
2,data_source,VARCHAR,NO,PRI,None,None
3,fetch_timestamp,TIMESTAMP,YES,None,None,None
4,record_count,INTEGER,YES,None,None,None
5,status,VARCHAR,YES,None,None,None


,n
0,114


scanner_daily_metrics


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,symbol,VARCHAR,YES,None,None,None
2,short_volume,DOUBLE,YES,None,None,None
3,short_exempt_volume,DOUBLE,YES,None,None,None
4,total_volume,DOUBLE,YES,None,None,None
5,short_total_volume,DOUBLE,YES,None,None,None
6,short_buy_volume,DOUBLE,YES,None,None,None
7,short_sell_volume,DOUBLE,YES,None,None,None
8,short_ratio,DOUBLE,YES,None,None,None
9,short_ratio_z,DOUBLE,YES,None,None,None


,n
0,0


In [26]:
sample_symbol = "META"
start_date = min(config.target_dates)
end_date = max(config.target_dates)
sample_symbol, start_date, end_date


('META', datetime.date(2025, 11, 20), datetime.date(2025, 12, 31))

## Raw Tables Samples


In [6]:
conn.execute("""
SELECT * FROM finra_otc_weekly_raw
WHERE symbol = ?
ORDER BY week_start_date DESC
LIMIT 5
""", [sample_symbol]).df()


,symbol,week_start_date,off_exchange_volume,trade_count,tier_identifier,tier_description,issue_name,market_participant_name,mpid,last_update_date,source_file


In [7]:
conn.execute("""
SELECT * FROM finra_short_daily_raw
WHERE symbol = ? AND trade_date BETWEEN ? AND ?
ORDER BY trade_date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,trade_date,short_volume,short_exempt_volume,total_volume,market,source,source_file


In [8]:
conn.execute("""
SELECT * FROM polygon_daily_agg_raw
WHERE symbol = ? AND trade_date BETWEEN ? AND ?
ORDER BY trade_date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,trade_date,open,high,low,close,vwap,volume,fetch_timestamp


In [9]:
conn.execute("""
SELECT * FROM polygon_equity_trades_raw
WHERE symbol = ? AND timestamp::DATE BETWEEN ? AND ?
ORDER BY timestamp DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,timestamp,price,size,bid,ask,data_source


## Derived Tables Samples


In [10]:
conn.execute("""
SELECT * FROM lit_direction_daily
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


,symbol,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell,classification_method,lit_coverage_pct,inference_version


In [11]:
conn.execute("""
SELECT * FROM daily_metrics
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
LIMIT 10
""", [sample_symbol, start_date, end_date]).df()


,date,symbol,log_buy_sell,short_volume,short_exempt_volume,short_total_volume,short_buy_volume,short_sell_volume,short_ratio,short_ratio_z,...,accumulation_score_display,confidence,data_quality,has_otc,has_short,has_lit,has_price,pressure_context_label,inference_version,accumulation_short_z_source


In [12]:
conn.execute("""
SELECT index_symbol, trade_date, coverage_count, expected_constituent_count, coverage_pct
FROM index_constituent_short_agg_daily
ORDER BY trade_date DESC
LIMIT 5
""").df()


,index_symbol,trade_date,coverage_count,expected_constituent_count,coverage_pct


## Coverage and Provenance Checks


In [13]:
conn.execute("""
SELECT symbol, COUNT(*) AS rows
FROM daily_metrics
WHERE date BETWEEN ? AND ?
GROUP BY symbol
ORDER BY symbol
""", [start_date, end_date]).df()


,symbol,rows


In [14]:
conn.execute("""
SELECT date, symbol, otc_off_exchange_volume, otc_week_used, data_quality
FROM daily_metrics
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
""", [sample_symbol, start_date, end_date]).df()


,date,symbol,otc_off_exchange_volume,otc_week_used,data_quality


## Sanity Checks


In [15]:
conn.execute("""
SELECT symbol, date, lit_buy_volume, lit_sell_volume, log_buy_sell
FROM lit_direction_daily
WHERE (lit_buy_volume <= 0 OR lit_sell_volume <= 0)
  AND log_buy_sell IS NOT NULL
LIMIT 10
""").df()


,symbol,date,lit_buy_volume,lit_sell_volume,log_buy_sell


In [16]:
conn.execute("""
SELECT symbol, date, short_ratio, short_ratio_denominator_type
FROM daily_metrics
WHERE short_ratio_denominator_type IS NOT NULL
  AND (short_ratio < 0 OR short_ratio > 1)
LIMIT 10
""").df()


,symbol,date,short_ratio,short_ratio_denominator_type


In [17]:
conn.execute("""
SELECT symbol, date, COUNT(*) AS n
FROM daily_metrics
GROUP BY symbol, date
HAVING n > 1
""").df()


,symbol,date,n


In [18]:
conn.execute("""
SELECT symbol, date, COUNT(*) AS n
FROM lit_direction_daily
GROUP BY symbol, date
HAVING n > 1
""").df()


,symbol,date,n


## Lit Data Diagnostics
Checking lit_direction_daily and daily_metrics for lit flow issues.

In [19]:
# 1. Check date range in lit_direction_daily (overall)
print("=== Date range in lit_direction_daily ===")
display(conn.execute("""
    SELECT MIN(date) as min_date, MAX(date) as max_date, COUNT(*) as total_rows
    FROM lit_direction_daily
""").df())

=== Date range in lit_direction_daily ===


,min_date,max_date,total_rows
0,2025-11-20,2025-12-31,58


In [20]:
# 2. Check which symbols have lit data
print("=== Symbols with lit data (lit_direction_daily) ===")
display(conn.execute("""
    SELECT symbol, COUNT(*) as rows, MIN(date) as min_date, MAX(date) as max_date
    FROM lit_direction_daily
    GROUP BY symbol
    ORDER BY rows DESC
""").df())

=== Symbols with lit data (lit_direction_daily) ===


,symbol,rows,min_date,max_date
0,NKE,29,2025-11-20,2025-12-31
1,META,29,2025-11-20,2025-12-31


In [21]:
# 3. Check lit_direction_daily for sample symbols (AAPL, SPY, XLF)
print("=== Lit data for sample symbols ===")
for sym in ["AAPL", "SPY", "XLF"]:
    print(f"\n--- {sym} ---")
    display(conn.execute("""
        SELECT date, lit_buy_volume, lit_sell_volume, lit_buy_ratio, log_buy_sell
        FROM lit_direction_daily
        WHERE symbol = ?
        ORDER BY date DESC LIMIT 10
    """, [sym]).df())

=== Lit data for sample symbols ===

--- AAPL ---


,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell



--- SPY ---


,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell



--- XLF ---


,date,lit_buy_volume,lit_sell_volume,lit_buy_ratio,log_buy_sell


In [22]:
# 4. Check daily_metrics lit columns for sample symbols
print("=== Daily_metrics lit columns for sample symbols ===")
for sym in ["AAPL", "SPY", "XLF"]:
    print(f"\n--- {sym} ---")
    display(conn.execute("""
        SELECT date, lit_flow_imbalance, lit_buy_ratio, lit_buy_volume, lit_sell_volume
        FROM daily_metrics
        WHERE symbol = ?
        ORDER BY date DESC LIMIT 10
    """, [sym]).df())

=== Daily_metrics lit columns for sample symbols ===

--- AAPL ---


,date,lit_flow_imbalance,lit_buy_ratio,lit_buy_volume,lit_sell_volume



--- SPY ---


,date,lit_flow_imbalance,lit_buy_ratio,lit_buy_volume,lit_sell_volume



--- XLF ---


,date,lit_flow_imbalance,lit_buy_ratio,lit_buy_volume,lit_sell_volume


In [23]:
# 5. Compare config target_dates with DB date ranges
print("=== Current config target_dates ===")
print(f"Min: {min(config.target_dates)}, Max: {max(config.target_dates)}")
print(f"Count: {len(config.target_dates)}")

print("\n=== Overlap check: lit_direction_daily dates within target range ===")
display(conn.execute("""
    SELECT COUNT(*) as matching_rows
    FROM lit_direction_daily
    WHERE date >= ? AND date <= ?
""", [min(config.target_dates), max(config.target_dates)]).df())

=== Current config target_dates ===
Min: 2025-11-20, Max: 2025-12-31
Count: 30

=== Overlap check: lit_direction_daily dates within target range ===


,matching_rows
0,58


In [24]:
# 6. Check polygon_ingestion_state (caching)
print("=== Polygon ingestion state (cached trades) ===")
display(conn.execute("""
    SELECT symbol, data_source, COUNT(*) as cached_dates, 
           MIN(trade_date) as min_date, MAX(trade_date) as max_date
    FROM polygon_ingestion_state
    GROUP BY symbol, data_source
    ORDER BY symbol
""").df())

=== Polygon ingestion state (cached trades) ===


,symbol,data_source,cached_dates,min_date,max_date
0,META,daily,28,2025-11-20,2025-12-31
1,META,tick,29,2025-11-20,2025-12-31
2,NKE,tick,29,2025-11-20,2025-12-31
3,NKE,daily,28,2025-11-20,2025-12-31


In [25]:
conn.close()
